In [2]:
#注意事项:
#当运行本Notebook的程序后，如果要关闭Notebook，请选择菜单: File > Close and Halt 才能确实停止当前正在运行的程序，并且释放资源
#如果没有使用以上方法，只关闭此分页，程序仍在运行，未释放资源，当您打开并运行其他的Notebook，可能会发生错误

# 20	Spark ML Pipeline 二元分类机器学习

In [3]:
sc.master

u'spark://master:7077'

# 20.1	数据准备

In [4]:
global Path    
if sc.master[0:5]=="local" :
   Path="file:/home/hduser/pythonsparkexample/PythonProject/"
else:   
   Path="hdfs://master:9000/user/hduser/"
#如果要在cluster模式运行(hadoop yarn 或Spark Stand alone)，请按照书上的说明，先把文件上传到HDFS目录

In [5]:
row_df = sqlContext.read.format("csv") \
     .option("header", "true") \
     .option("delimiter", "\t") \
     .load(Path+"data/train.tsv")
print row_df.count()    

7395


In [6]:
row_df.printSchema()

root
 |-- url: string (nullable = true)
 |-- urlid: string (nullable = true)
 |-- boilerplate: string (nullable = true)
 |-- alchemy_category: string (nullable = true)
 |-- alchemy_category_score: string (nullable = true)
 |-- avglinksize: string (nullable = true)
 |-- commonlinkratio_1: string (nullable = true)
 |-- commonlinkratio_2: string (nullable = true)
 |-- commonlinkratio_3: string (nullable = true)
 |-- commonlinkratio_4: string (nullable = true)
 |-- compression_ratio: string (nullable = true)
 |-- embed_ratio: string (nullable = true)
 |-- framebased: string (nullable = true)
 |-- frameTagRatio: string (nullable = true)
 |-- hasDomainLink: string (nullable = true)
 |-- html_ratio: string (nullable = true)
 |-- image_ratio: string (nullable = true)
 |-- is_news: string (nullable = true)
 |-- lengthyLinkDomain: string (nullable = true)
 |-- linkwordscore: string (nullable = true)
 |-- news_front_page: string (nullable = true)
 |-- non_markup_alphanum_characters: string (nulla

In [7]:
row_df.select('url','alchemy_category','alchemy_category_score','is_news','label').show(10)

+--------------------+------------------+----------------------+-------+-----+
|                 url|  alchemy_category|alchemy_category_score|is_news|label|
+--------------------+------------------+----------------------+-------+-----+
|http://www.bloomb...|          business|              0.789131|      1|    0|
|http://www.popsci...|        recreation|              0.574147|      1|    1|
|http://www.menshe...|            health|              0.996526|      1|    1|
|http://www.dumbli...|            health|              0.801248|      1|    1|
|http://bleacherre...|            sports|              0.719157|      1|    0|
|http://www.conven...|                 ?|                     ?|      ?|    0|
|http://gofashionl...|arts_entertainment|               0.22111|      1|    1|
|http://www.inside...|                 ?|                     ?|      ?|    0|
|http://www.valetm...|                 ?|                     ?|      1|    1|
|http://www.howswe...|                 ?|           

In [8]:
from pyspark.sql.functions import udf
def replace_question(x):
    return ("0" if x=="?" else x)
replace_question= udf(replace_question)

In [9]:
from pyspark.sql.functions import col  
import pyspark.sql.types 

In [10]:
df= row_df.select(
               ['url','alchemy_category' ]+
               [replace_question(col(column)).cast("double").alias(column)  
                for column in row_df.columns[4:] ] )

In [11]:
print df.printSchema()

root
 |-- url: string (nullable = true)
 |-- alchemy_category: string (nullable = true)
 |-- alchemy_category_score: double (nullable = true)
 |-- avglinksize: double (nullable = true)
 |-- commonlinkratio_1: double (nullable = true)
 |-- commonlinkratio_2: double (nullable = true)
 |-- commonlinkratio_3: double (nullable = true)
 |-- commonlinkratio_4: double (nullable = true)
 |-- compression_ratio: double (nullable = true)
 |-- embed_ratio: double (nullable = true)
 |-- framebased: double (nullable = true)
 |-- frameTagRatio: double (nullable = true)
 |-- hasDomainLink: double (nullable = true)
 |-- html_ratio: double (nullable = true)
 |-- image_ratio: double (nullable = true)
 |-- is_news: double (nullable = true)
 |-- lengthyLinkDomain: double (nullable = true)
 |-- linkwordscore: double (nullable = true)
 |-- news_front_page: double (nullable = true)
 |-- non_markup_alphanum_characters: double (nullable = true)
 |-- numberOfLinks: double (nullable = true)
 |-- numwords_in_url: d

In [12]:
df.select('url','alchemy_category','alchemy_category_score','is_news','label').show(10)

+--------------------+------------------+----------------------+-------+-----+
|                 url|  alchemy_category|alchemy_category_score|is_news|label|
+--------------------+------------------+----------------------+-------+-----+
|http://www.bloomb...|          business|              0.789131|    1.0|  0.0|
|http://www.popsci...|        recreation|              0.574147|    1.0|  1.0|
|http://www.menshe...|            health|              0.996526|    1.0|  1.0|
|http://www.dumbli...|            health|              0.801248|    1.0|  1.0|
|http://bleacherre...|            sports|              0.719157|    1.0|  0.0|
|http://www.conven...|                 ?|                   0.0|    0.0|  0.0|
|http://gofashionl...|arts_entertainment|               0.22111|    1.0|  1.0|
|http://www.inside...|                 ?|                   0.0|    0.0|  0.0|
|http://www.valetm...|                 ?|                   0.0|    1.0|  1.0|
|http://www.howswe...|                 ?|           

In [13]:
train_df, test_df = df.randomSplit([0.7, 0.3])
train_df.cache()
test_df.cache()

DataFrame[url: string, alchemy_category: string, alchemy_category_score: double, avglinksize: double, commonlinkratio_1: double, commonlinkratio_2: double, commonlinkratio_3: double, commonlinkratio_4: double, compression_ratio: double, embed_ratio: double, framebased: double, frameTagRatio: double, hasDomainLink: double, html_ratio: double, image_ratio: double, is_news: double, lengthyLinkDomain: double, linkwordscore: double, news_front_page: double, non_markup_alphanum_characters: double, numberOfLinks: double, numwords_in_url: double, parametrizedLinkRatio: double, spelling_errors_ratio: double, label: double]

# 20.2	介绍数据处理的pipeline管线的组件

In [14]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import  StringIndexer, OneHotEncoder,VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier

In [15]:
from pyspark.ml.feature import  StringIndexer

In [16]:
categoryIndexer = StringIndexer(
                                   inputCol='alchemy_category', 
                                   outputCol="alchemy_category_Index")

In [17]:
categoryTransformer=categoryIndexer.fit(df)

In [18]:
for i in range(0,len(categoryTransformer.labels)):
    print str(i)+':'+categoryTransformer.labels[i] 

0:?
1:recreation
2:arts_entertainment
3:business
4:health
5:sports
6:culture_politics
7:computer_internet
8:science_technology
9:gaming
10:religion
11:law_crime
12:unknown
13:weather


In [19]:
df1=categoryTransformer.transform(df)

In [20]:
print df1.columns

['url', 'alchemy_category', 'alchemy_category_score', 'avglinksize', 'commonlinkratio_1', 'commonlinkratio_2', 'commonlinkratio_3', 'commonlinkratio_4', 'compression_ratio', 'embed_ratio', 'framebased', 'frameTagRatio', 'hasDomainLink', 'html_ratio', 'image_ratio', 'is_news', 'lengthyLinkDomain', 'linkwordscore', 'news_front_page', 'non_markup_alphanum_characters', 'numberOfLinks', 'numwords_in_url', 'parametrizedLinkRatio', 'spelling_errors_ratio', 'label', 'alchemy_category_Index']


In [21]:
df1.select("alchemy_category","alchemy_category_Index").show(10)

+------------------+----------------------+
|  alchemy_category|alchemy_category_Index|
+------------------+----------------------+
|          business|                   3.0|
|        recreation|                   1.0|
|            health|                   4.0|
|            health|                   4.0|
|            sports|                   5.0|
|                 ?|                   0.0|
|arts_entertainment|                   2.0|
|                 ?|                   0.0|
|                 ?|                   0.0|
|                 ?|                   0.0|
+------------------+----------------------+
only showing top 10 rows



In [22]:
from pyspark.ml.feature import  OneHotEncoder

In [23]:
encoder = OneHotEncoder(dropLast=False,
                            inputCol='alchemy_category_Index', 
                            outputCol="alchemy_category_IndexVec")

In [24]:
df2=encoder.transform(df1)
print df2.columns

['url', 'alchemy_category', 'alchemy_category_score', 'avglinksize', 'commonlinkratio_1', 'commonlinkratio_2', 'commonlinkratio_3', 'commonlinkratio_4', 'compression_ratio', 'embed_ratio', 'framebased', 'frameTagRatio', 'hasDomainLink', 'html_ratio', 'image_ratio', 'is_news', 'lengthyLinkDomain', 'linkwordscore', 'news_front_page', 'non_markup_alphanum_characters', 'numberOfLinks', 'numwords_in_url', 'parametrizedLinkRatio', 'spelling_errors_ratio', 'label', 'alchemy_category_Index', 'alchemy_category_IndexVec']


In [25]:
df2.select("alchemy_category","alchemy_category_Index",
                    "alchemy_category_IndexVec").show(10)

+------------------+----------------------+-------------------------+
|  alchemy_category|alchemy_category_Index|alchemy_category_IndexVec|
+------------------+----------------------+-------------------------+
|          business|                   3.0|           (14,[3],[1.0])|
|        recreation|                   1.0|           (14,[1],[1.0])|
|            health|                   4.0|           (14,[4],[1.0])|
|            health|                   4.0|           (14,[4],[1.0])|
|            sports|                   5.0|           (14,[5],[1.0])|
|                 ?|                   0.0|           (14,[0],[1.0])|
|arts_entertainment|                   2.0|           (14,[2],[1.0])|
|                 ?|                   0.0|           (14,[0],[1.0])|
|                 ?|                   0.0|           (14,[0],[1.0])|
|                 ?|                   0.0|           (14,[0],[1.0])|
+------------------+----------------------+-------------------------+
only showing top 10 

In [26]:
from pyspark.ml.feature import  VectorAssembler

In [27]:
assemblerInputs =['alchemy_category_IndexVec']  +  row_df.columns[4:-1]
print assemblerInputs

['alchemy_category_IndexVec', 'alchemy_category_score', 'avglinksize', 'commonlinkratio_1', 'commonlinkratio_2', 'commonlinkratio_3', 'commonlinkratio_4', 'compression_ratio', 'embed_ratio', 'framebased', 'frameTagRatio', 'hasDomainLink', 'html_ratio', 'image_ratio', 'is_news', 'lengthyLinkDomain', 'linkwordscore', 'news_front_page', 'non_markup_alphanum_characters', 'numberOfLinks', 'numwords_in_url', 'parametrizedLinkRatio', 'spelling_errors_ratio']


In [28]:
assembler = VectorAssembler(
                            inputCols=assemblerInputs,  
                            outputCol="features")

In [29]:
df3=assembler.transform(df2)

In [30]:
print df3.columns

['url', 'alchemy_category', 'alchemy_category_score', 'avglinksize', 'commonlinkratio_1', 'commonlinkratio_2', 'commonlinkratio_3', 'commonlinkratio_4', 'compression_ratio', 'embed_ratio', 'framebased', 'frameTagRatio', 'hasDomainLink', 'html_ratio', 'image_ratio', 'is_news', 'lengthyLinkDomain', 'linkwordscore', 'news_front_page', 'non_markup_alphanum_characters', 'numberOfLinks', 'numwords_in_url', 'parametrizedLinkRatio', 'spelling_errors_ratio', 'label', 'alchemy_category_Index', 'alchemy_category_IndexVec', 'features']


In [31]:
df3.select('features').show(5)

+--------------------+
|            features|
+--------------------+
|(36,[3,14,15,16,1...|
|(36,[1,14,15,16,1...|
|(36,[4,14,15,16,1...|
|(36,[4,14,15,16,1...|
|(36,[5,14,15,16,1...|
+--------------------+
only showing top 5 rows



In [32]:
df3.select('features').take(1)

[Row(features=SparseVector(36, {3: 1.0, 14: 0.7891, 15: 2.0556, 16: 0.6765, 17: 0.2059, 18: 0.0471, 19: 0.0235, 20: 0.4438, 23: 0.0908, 25: 0.2458, 26: 0.0039, 27: 1.0, 28: 1.0, 29: 24.0, 31: 5424.0, 32: 170.0, 33: 8.0, 34: 0.1529, 35: 0.0791}))]

In [33]:
from pyspark.ml.classification import DecisionTreeClassifier

In [34]:
dt = DecisionTreeClassifier(labelCol="label",  featuresCol="features",
                                              impurity="gini",maxDepth=10, maxBins=14)

In [35]:
dt_model=dt.fit(df3)
print dt_model

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_47bfbe3b2a9b0806f2f4) of depth 10 with 723 nodes


In [36]:
df4=dt_model.transform(df3)

# 20.3	建立数据处理的pipeline管线

In [37]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import  StringIndexer, OneHotEncoder,VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier

In [38]:
stringIndexer = StringIndexer(inputCol='alchemy_category', 
                                                outputCol="alchemy_category_Index")
encoder = OneHotEncoder(dropLast=False,
                                            inputCol='alchemy_category_Index',
                                            outputCol="alchemy_category_IndexVec")
assemblerInputs =['alchemy_category_IndexVec']  + row_df.columns[4:-1] 
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features",impurity="gini",
                                                   maxDepth=10, maxBins=14)
pipeline = Pipeline(stages=[stringIndexer,encoder ,assembler,dt ])

In [39]:
pipeline.getStages()

[StringIndexer_4b34ad72ca393b323196,
 OneHotEncoder_4dbdbf1b40995af06d47,
 VectorAssembler_42f0a4cf10ed2b1f2406,
 DecisionTreeClassifier_4b07b70db603702b8a56]

# 20.4	使用pipeline进行训练

In [40]:
pipelineModel = pipeline.fit(train_df)

In [41]:
pipelineModel.stages[3]

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4b07b70db603702b8a56) of depth 10 with 477 nodes

In [42]:
print pipelineModel.stages[3].toDebugString[:1000]

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4b07b70db603702b8a56) of depth 10 with 477 nodes
  If (feature 31 <= 1309.0)
   If (feature 23 <= 0.035783366)
    If (feature 16 <= 0.757255937)
     If (feature 32 <= 50.0)
      If (feature 9 in {1.0})
       Predict: 0.0
      Else (feature 9 not in {1.0})
       If (feature 4 in {1.0})
        Predict: 1.0
       Else (feature 4 not in {1.0})
        If (feature 15 <= 2.76744186)
         If (feature 35 <= 0.116883117)
          If (feature 35 <= 0.077970297)
           If (feature 35 <= 0.064676617)
            Predict: 1.0
           Else (feature 35 > 0.064676617)
            Predict: 0.0
          Else (feature 35 > 0.077970297)
           If (feature 33 <= 0.0)
            Predict: 0.0
           Else (feature 33 > 0.0)
            Predict: 1.0
         Else (feature 35 > 0.116883117)
          If (feature 29 <= 25.0)
           If (feature 15 <= 1.540540541)
            Predict: 1.0
           Else (feature 15 > 1.5

# 20.5	使用pipeline 进行预测

In [43]:
predicted=pipelineModel.transform(test_df)

In [44]:
predicted.columns

['url',
 'alchemy_category',
 'alchemy_category_score',
 'avglinksize',
 'commonlinkratio_1',
 'commonlinkratio_2',
 'commonlinkratio_3',
 'commonlinkratio_4',
 'compression_ratio',
 'embed_ratio',
 'framebased',
 'frameTagRatio',
 'hasDomainLink',
 'html_ratio',
 'image_ratio',
 'is_news',
 'lengthyLinkDomain',
 'linkwordscore',
 'news_front_page',
 'non_markup_alphanum_characters',
 'numberOfLinks',
 'numwords_in_url',
 'parametrizedLinkRatio',
 'spelling_errors_ratio',
 'label',
 'alchemy_category_Index',
 'alchemy_category_IndexVec',
 'features',
 'rawPrediction',
 'probability',
 'prediction']

In [45]:
predicted.select('url','features','rawprediction','probability','label','prediction').show(10)

+--------------------+--------------------+-------------+--------------------+-----+----------+
|                 url|            features|rawprediction|         probability|label|prediction|
+--------------------+--------------------+-------------+--------------------+-----+----------+
|http://1000awesom...|(36,[0,15,16,17,1...| [96.0,339.0]|[0.22068965517241...|  1.0|       1.0|
|http://30aeats.co...|(36,[0,15,16,17,1...|  [12.0,24.0]|[0.33333333333333...|  1.0|       1.0|
|http://3kidsandus...|(36,[0,15,16,17,1...| [61.0,151.0]|[0.28773584905660...|  0.0|       1.0|
|http://3kidsandus...|(36,[0,15,16,17,1...|  [12.0,24.0]|[0.33333333333333...|  1.0|       1.0|
|http://6jokes.com...|(36,[3,14,15,16,1...|   [68.0,8.0]|[0.89473684210526...|  0.0|       0.0|
|http://9gg.us/hah...|(36,[12,14,15,20,...|   [20.0,2.0]|[0.90909090909090...|  0.0|       0.0|
|http://9gg.us/jus...|(36,[2,14,15,16,2...|   [35.0,0.0]|           [1.0,0.0]|  0.0|       0.0|
|http://abeautiful...|(36,[1,14,15,16,1.

In [46]:
predicted.select('probability','prediction') .take(10)

[Row(probability=DenseVector([0.2207, 0.7793]), prediction=1.0),
 Row(probability=DenseVector([0.3333, 0.6667]), prediction=1.0),
 Row(probability=DenseVector([0.2877, 0.7123]), prediction=1.0),
 Row(probability=DenseVector([0.3333, 0.6667]), prediction=1.0),
 Row(probability=DenseVector([0.8947, 0.1053]), prediction=0.0),
 Row(probability=DenseVector([0.9091, 0.0909]), prediction=0.0),
 Row(probability=DenseVector([1.0, 0.0]), prediction=0.0),
 Row(probability=DenseVector([0.1197, 0.8803]), prediction=1.0),
 Row(probability=DenseVector([0.2207, 0.7793]), prediction=1.0),
 Row(probability=DenseVector([0.511, 0.489]), prediction=0.0)]

# 20.6	评估模型的准确率

In [47]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [48]:
evaluator = BinaryClassificationEvaluator(
                              rawPredictionCol="rawPrediction",
                              labelCol="label",  
                              metricName="areaUnderROC"  )

In [49]:
predictions =pipelineModel.transform(test_df)
auc= evaluator.evaluate(predictions)
auc

0.6370924405873424

# 20.7	使用TrainValidation进行训练评估找出最佳模型

In [50]:
from pyspark.ml.tuning import ParamGridBuilder,TrainValidationSplit

In [51]:
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")

In [52]:
paramGrid = ParamGridBuilder()\
  .addGrid(dt.impurity, [ "gini","entropy"])\
  .addGrid(dt.maxDepth, [ 5,10,15])\
  .addGrid(dt.maxBins, [10, 15,20])\
  .build()

In [53]:
tvs = TrainValidationSplit(estimator=dt,evaluator=evaluator,
                  estimatorParamMaps=paramGrid,trainRatio=0.8)

In [54]:
tvs_pipeline = Pipeline(stages=[stringIndexer,encoder ,assembler, tvs])

In [55]:
tvs_pipelineModel =tvs_pipeline.fit(train_df)

In [56]:
bestModel=tvs_pipelineModel.stages[3].bestModel
bestModel

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4956af220acb83b94d62) of depth 15 with 1637 nodes

In [57]:
print bestModel.toDebugString[:500]

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4956af220acb83b94d62) of depth 15 with 1637 nodes
  If (feature 31 <= 1213.0)
   If (feature 23 <= 0.038626609)
    If (feature 2 in {1.0})
     If (feature 35 <= 0.097633136)
      If (feature 25 <= 0.25600839)
       Predict: 0.0
      Else (feature 25 > 0.25600839)
       If (feature 33 <= 3.0)
        Predict: 1.0
       Else (feature 33 > 3.0)
        Predict: 0.0
     Else (feature 35 > 0.097633136)
      If (feature 15 <= 2.28205


In [58]:
predictions = tvs_pipelineModel.transform(test_df)
auc= evaluator.evaluate(predictions)
auc

0.6571614250977336

# 20.8	使用crossValidation进行训练评估找出最佳模型

In [59]:
from pyspark.ml.tuning import CrossValidator

In [60]:
cv = CrossValidator(estimator=dt, evaluator=evaluator, 
                    estimatorParamMaps=paramGrid, numFolds=3)

In [61]:
cv_pipeline = Pipeline(stages=[stringIndexer,encoder ,assembler, cv])

In [62]:
cv_pipelineModel = cv_pipeline.fit(train_df)

In [63]:
bestModel=cv_pipelineModel.stages[3].bestModel
bestModel

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4956af220acb83b94d62) of depth 15 with 1657 nodes

In [64]:
print bestModel.toDebugString[:500]

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4956af220acb83b94d62) of depth 15 with 1657 nodes
  If (feature 31 <= 1213.0)
   If (feature 23 <= 0.034250421)
    If (feature 16 <= 0.763157895)
     If (feature 20 <= 0.46797153)
      If (feature 26 <= 0.923076923)
       If (feature 18 <= 0.048034934)
        Predict: 0.0
       Else (feature 18 > 0.048034934)
        If (feature 19 <= 0.013100437)
         If (feature 27 <= 0.0)
          Predict: 1.0
         Else (feature 27 > 0


In [65]:
predictions = cv_pipelineModel.transform(test_df)
auc= evaluator.evaluate(predictions)
auc

0.6644335142469471

# 20.9	使用RandomForestClassifier分类器

In [66]:
from pyspark.ml.classification import RandomForestClassifier

rf =RandomForestClassifier(labelCol="label", 
                           featuresCol="features",numTrees=10)

rfpipeline = Pipeline(stages=[stringIndexer,encoder ,assembler,rf ])

In [67]:
rfpipelineModel = rfpipeline.fit(train_df)
rfpredicted=rfpipelineModel.transform(test_df)
evaluator.evaluate(rfpredicted)

0.7224742063758629

In [68]:
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier

paramGrid = ParamGridBuilder()\
  .addGrid(rf.impurity, [ "gini","entropy"])\
  .addGrid(rf.maxDepth, [ 5,10,15])\
  .addGrid(rf.maxBins, [10, 15,20])\
  .addGrid(rf.numTrees, [10, 20,30])\
  .build()

rftvs = TrainValidationSplit(estimator=rf, evaluator=evaluator,
                                 estimatorParamMaps=paramGrid, trainRatio=0.8)

rftvs_pipeline = Pipeline(stages=[stringIndexer,encoder ,assembler, rftvs])
rftvs_pipelineModel =rftvs_pipeline.fit(train_df)
rftvspredictions = rftvs_pipelineModel.transform(test_df)
auc= evaluator.evaluate(rftvspredictions)
auc

0.7566620934481516

In [69]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml import Pipeline

rfcv = CrossValidator(estimator=rf, evaluator=evaluator, 
                          estimatorParamMaps=paramGrid, numFolds=3)

rfcv_pipeline = Pipeline(stages=[stringIndexer,encoder ,assembler, rfcv])
rfcv_pipelineModel = rfcv_pipeline.fit(train_df)

In [70]:
rfcvpredictions = rfcv_pipelineModel.transform(test_df)

In [71]:
DescDict = {
           0: "暂时性网页(ephemeral)",
           1: "长青网页(evergreen)"
     }
for data in rfcvpredictions .select('url','prediction').take(5):
    print " 网址：  " +str(data[0])+"\n" +\
                  "             ==>预测:"+ str(data[1])+ \
                  " 说明:"+DescDict[data[1]] +"\n"

 网址：  http://1000awesomethings.com/2008/07/07/989-blowing-your-nose-in-the-shower/
             ==>预测:1.0 说明:长青网页(evergreen)

 网址：  http://30aeats.com/recipes/sun-dried-tomato-kalamata-olive-and-basil-pesto-focaccia/
             ==>预测:1.0 说明:长青网页(evergreen)

 网址：  http://3kidsandus.com/2011/de-stress-me-dessert-black-forest-brownie-bites/
             ==>预测:1.0 说明:长青网页(evergreen)

 网址：  http://3kidsandus.com/red-velvet-rice-krispies-treats-hearts-for-valentines-day/
             ==>预测:1.0 说明:长青网页(evergreen)

 网址：  http://6jokes.com/just-hanging-on-the-wall/
             ==>预测:0.0 说明:暂时性网页(ephemeral)



In [72]:
auc= evaluator.evaluate(rfcvpredictions)
auc

0.7566620934481524